In [1]:
import sys
from PyQt4.QtCore import pyqtSlot
from PyQt4 import QtGui, QtCore
import glob, openpyxl, pandas as pd, numpy as np
from datetime import datetime, timedelta

class Window(QtGui.QMainWindow):

    def __init__(self):
        
        super(Window, self).__init__()
        self.setGeometry(50, 50, 500, 700)
        self.setWindowTitle("RMR Calculator")

        extractAction = QtGui.QAction("Quit", self)
        extractAction.setShortcut("Ctrl+Q")
        extractAction.setStatusTip('Quit The App')
        extractAction.triggered.connect(self.close)

        self.statusBar()

        mainMenu = self.menuBar()
        
        fileMenu = mainMenu.addMenu('&File')
        fileMenu.addAction(extractAction)

        self.home()

    def home(self):
        
        # Create textbox
        self.textbox = QtGui.QLineEdit(self)
        self.textbox.move(130,80)
        self.textbox.resize(300, 20)
        
        # Create a button in the window
        self.btn = QtGui.QPushButton("Select File", self)
        self.btn.clicked.connect(self.select_file)
        self.btn.move(20,80)
        self.btn.resize(100,20)
        
        self.btn = QtGui.QPushButton("Calculate RMR", self)
        self.btn.clicked.connect(self.calculate_RMR)
        self.btn.move(320, 120)
        self.btn.resize(100,20)
        
        self.show()

    def select_file(self):
        fileName = QtGui.QFileDialog.getOpenFileName(self, 'Select File','*.xlsx')
        self.textbox.setText(fileName)
    
    def calculate_RMR(self):
        
        def create_dataframe():
            columns = ['Patient ID', 'Age', 'Sex','Height (in)', 'Height (cm)', 'Weight (Ib)',
           'Weight (kg)', 'Time (min)', 'VO2/kg STPD (ml/kg/m)', 'VO2 STPD (L/min)',
          '(ml/min)', 'VCO2 STPD (L/min)', '(ml/min)', 'VE STPD (L/min)', 'RQ', 
           'RR (BPM)', 'VE/VO2 (BT/ST)', 'METS', 'TM SPD(mph)', 'Start','End']
            df = pd.DataFrame(columns = columns)
            return df

        filename = self.textbox.text()
        print(filename)
        if filename:
            print("if")
            df = create_dataframe()
            print(df)
        else:
            msg = QtGui.QMessageBox()
            msg.setInformativeText("You have to select the data file first in order to calculate.")
            msg.setWindowTitle("Warning")
            msg.setStandardButtons(QMessageBox.OK)
            msg.buttonClicked.connect(msg.close)
    

def run():
    app = QtGui.QApplication(sys.argv)
    GUI = Window()
    sys.exit(app.exec_())


run()     

NameError: name 'QMessageBox' is not defined

SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.
